<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/note_books/lgbm_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import shutil
import scipy.stats as stats
from pathlib import Path
import glob
from tqdm import tqdm


from sklearn.model_selection import KFold
import lightgbm as lgb

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager

from sklearn.preprocessing import LabelEncoder

In [2]:
N_SPLITS = 10
SEED = 42


In [3]:
LOG_PATH = Path("./log/")
LOG_PATH.mkdir(parents=True, exist_ok=True)


In [4]:
@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


In [5]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    
def comp_metric(xhat, yhat, x, y):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2))
    return intermediate.sum()/xhat.shape[0]


def score_log(df: pd.DataFrame, num_files: int, nam_file: str, data_shape: tuple, n_fold: int, seed: int, mpe: float):
    score_dict = {'n_files': num_files, 'file_name': nam_file, 'shape': data_shape, 'fold': n_fold, 'seed': seed, 'score': mpe}
    # noinspection PyTypeChecker
    df = pd.concat([df, pd.DataFrame.from_dict([score_dict])])
    df.to_csv(LOG_PATH / f"log_score.csv", index=False)
    return df

In [6]:
set_seed(SEED)


In [7]:
feature_dir = "/content/drive/MyDrive/all_data"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
subm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv', index_col=0)

In [8]:
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.1,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

# 関数

In [37]:
def get_target_train(input_df):
    output_df = input_df.loc[:, ['x', 'y', 'path', 'timestamp' ]]
    return output_df


def get_numerical_features_train(input_df):
    output_df = input_df.drop(['beacon_uuid', 'beacon_major_id', 'beacon_minor_id', 'beacon_mac_addr',
                               'timestamp' ,'x', 'y', 'path'], axis=1)
    return output_df

def get_le_features(input_df):
    cat_cols = ['beacon_mac_addr']
    output_df = pd.DataFrame()
    for c in cat_cols:
        le = LabelEncoder()
        le.fit(input_df[c].astype(str))
        output_df["LE_"+c] = le.transform(input_df[c].astype(str))
    return output_df

def get_sum_sensor_features(input_df):
    output_df = pd.DataFrame()
    output_df['sum_sensor_x'] = input_df['acc_x'] + input_df['acc_unc_x'] + input_df['acc_unc_x2'] + input_df['gyro_x'] + input_df['gyro_unc_x'] + input_df['gyro_unc_x2'] + input_df['mag_x'] + input_df['mag_unc_x'] + input_df['mag_unc_x2'] + input_df['rotation_x']
    output_df['sum_sensor_y'] = input_df['acc_y'] + input_df['acc_unc_y'] + input_df['acc_unc_y2'] + input_df['gyro_y'] + input_df['gyro_unc_y'] + input_df['gyro_unc_y2'] + input_df['mag_y'] + input_df['mag_unc_y'] + input_df['mag_unc_y2'] + input_df['rotation_y']
    output_df['sum_sensor_xy'] = output_df['sum_sensor_x'] + output_df['sum_sensor_y']
    return output_df

In [38]:
def get_target_test(input_df):
    output_df = input_df.loc[:, ['site_path_timestamp', 'path', 'timestamp']]
    return output_df


def get_numerical_features_test(input_df):
    output_df = input_df.drop(['beacon_uuid', 'beacon_major_id', 'beacon_minor_id', 'beacon_mac_addr',
                               'site_path_timestamp', 'path', 'timestamp'], axis=1)
    return output_df

In [39]:
def get_process_funcs_train():
    funcs = [get_sum_sensor_features,
             get_numerical_features_train,
             get_target_train]
    return funcs

def get_process_funcs_test():
    funcs = [get_sum_sensor_features,
             get_numerical_features_test,
             get_target_test]
    return funcs

def to_feature(input_df, funcs):
    output_df = pd.DataFrame()
    for func in tqdm(funcs, total=len(funcs)):
        _df = func(input_df)
        assert len(_df) == len(input_df), func.__name__
        output_df = pd.concat([output_df, _df], axis=1)

    return output_df

In [49]:
score_df = pd.DataFrame()
oof = list()
predictions = list()
for n_files, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0).rename({'f':'floor'}, axis=1).fillna(0)
    test_data = pd.read_csv(test_files[n_files], index_col=0).fillna(0)

    process_funcs_train = get_process_funcs_train()
    process_funcs_test = get_process_funcs_test()
    data = to_feature(data, process_funcs_train)
    test_data = to_feature(test_data, process_funcs_test)

    print(data.shape)
    print(test_data.shape)

    oof_x, oof_y = np.zeros(data.shape[0]), np.zeros(data.shape[0])
    preds_x, preds_y = 0, 0

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
        X_train = data.iloc[trn_idx, :-4]
        y_trainx = data.iloc[trn_idx, -4]
        y_trainy = data.iloc[trn_idx, -3]

        X_valid = data.iloc[val_idx, :-4]
        y_validx = data.iloc[val_idx, -4]
        y_validy = data.iloc[val_idx, -3]
        
        modelx = lgb.LGBMRegressor(**lgb_params)
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       )

        modely = lgb.LGBMRegressor(**lgb_params)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       )

            
        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)

        preds_x += modelx.predict(test_data.iloc[:, :-3]) / N_SPLITS
        preds_y += modely.predict(test_data.iloc[:, :-3]) / N_SPLITS
        preds_f = test_data['floor'].values

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], y_validx.to_numpy(), y_validy.to_numpy())
        print(f"fold {fold}: mean position error {score}")
        score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, fold, SEED, score)

    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y,
                        data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy())
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)
    score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, 999, SEED, score)
    
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    test_preds.columns = subm.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)

100%|██████████| 3/3 [00:00<00:00, 112.58it/s]


(9296, 3438)
(299, 3437)


[13.9GB(+0.0GB): 17.532sec] fit X
[13.9GB(+0.0GB): 15.061sec] fit Y


fold 0: mean position error 3.52492257299057


[13.9GB(+0.0GB): 18.735sec] fit X
[13.9GB(+0.0GB): 17.341sec] fit Y


...

fold 7: mean position error 4.953713056648643


[12.0GB(+0.0GB): 24.021sec] fit X
[12.0GB(+0.0GB): 31.769sec] fit Y


fold 8: mean position error 5.203481128011933


[12.0GB(+0.0GB): 33.653sec] fit X
[12.0GB(+0.0GB): 28.007sec] fit Y


fold 9: mean position error 5.036929946824247
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
file #23, shape=(15655, 4905), name=5dc8cea7659e181adb076a3f_train.csv
mean position error 5.048670669657306
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+


In [46]:
data

,sum_sensor_x,sum_sensor_y,sum_sensor_xy,0006968a29229c78478c7f6c2d2d37ee6ba145ef,000adfcfbf793ddc4f9c6a1a7c3bc78cffd01819,0018258e7c0f5223eb1750ada8ae2a4c4c58d21c,002c92111c356bc9fc054042cd2f0a96d9e43b06,0040b8059620d2c17e22e24abb76a679200aba6a,0041e6526da83cbdd4b171648c7f172fc325be85,0094988edb57e8605962a80834f8d45cdb7b5664,00b110f7f41db9767115afc45013c37cfc6b7f98,00e6c8b25a32bf8ef89c491daae09ee845ac450c,00f583061184d29cfac0e03cf0982edd025e677a,010c689afe822af414b2b35793afc26f7601fbd6,01303b1e8f97b01bad8797e315ed0a901a329be4,0141e500290392d659a7626bab8583cf8dc38e29,014495aeff4f7fc100db0e678170a63572b4697e,015c7c64b6a54349e2397de357f568512772fb3e,016eded3ba71cdb8878fff411645997d03694957,018ce87fe87decc1e5774da5dc34635f1184511e,018e923b2cceeb208de55aafffd91952cad67095,01a1425d2c391b1ce611307c5d9e2f5aae9d05a9,01c0147829c7633d7d4a5ff0f0fb7b9dcae47ddb,01ee8bd0434333056d99c09516b87057dcde31af,01f8283d6a9c4569645891e427a13424a64dea19,01fc007de38446168c161ccc468b600c3ca264a5,021633338aefbc95c57277a06ac248446877d7dc,0217dcc23e4d2dc353049039969395a1ad8f37de,02245fd2e9a7c1ed386ec3d898e5f396e8f06b7d,0249b5890e0f0ddf780785a396beece6917181a3,027817c190697bc6fadec198a06b280359eb4fe8,0290599d79e3d93d92925711de912b784674843e,029ef8e3586e31fc20cf72262a4769d71ad26c2e,029f08c9ded6f97b2d9835fb82e70353e86d3030,02b3f57280dda5a50d93e8b0859602e47d9c9a71,02b474371945f487190faaf51a03428644540af8,02cd4f9d65034f697b3cab4e57b316348f86d915,030a4d88c65cc68d4df9ca64051fb9d76381f87e,03271f9864b63a30538e1b00fa711d2a3d1ac62b,0363d84f19fc2a4b9990694d94f56b4c99ef06db,...,ffbbc3e5b5569397a31ba6a6b40a282722f2d601,ffe57a1050c12fb060adae004f8b2a5ebe9ba9bd,beacon_tx_power,beacon_rssi,beacon_distance,acc_x,acc_y,acc_z,acc_unc_x,acc_unc_y,acc_unc_z,acc_unc_x2,acc_unc_y2,acc_unc_z2,gyro_x,gyro_y,gyro_z,gyro_unc_x,gyro_unc_y,gyro_unc_z,gyro_unc_x2,gyro_unc_y2,gyro_unc_z2,mag_x,mag_y,mag_z,mag_unc_x,mag_unc_y,mag_unc_z,mag_unc_x2,mag_unc_y2,mag_unc_z2,rotation_x,rotation_y,rotation_z,floor,x,y,path,timestamp
0,-39.686871,-119.635101,-159.321971,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-56.0,-81.0,15.596896,-0.929840,2.043625,11.139282,-0.961563,2.230362,11.060867,0.0,0.0,0.0,0.439972,0.067459,-0.032745,-0.081650,-0.128754,-0.048187,0.000000,0.000000,0.000000,22.070312,-15.106201,-20.103455,-60.224915,-108.815000,-354.11224,0.00000,0.00000,0.0000,0.000813,0.073408,0.931313,2,198.05206,93.971240,5d073b814a19c000086c558b,1560500997770
1,-27.997266,-123.018481,-151.015747,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-75.0,-57.0,0.064289,-1.628937,-1.168808,9.484283,-1.481705,-1.769745,12.809250,0.0,0.0,0.0,0.714279,-0.148270,-0.069489,0.254990,-0.183609,0.033310,0.000000,0.000000,0.000000,28.230286,-13.053894,-11.450195,-54.064940,-106.762695,-345.45898,0.00000,0.00000,0.0000,-0.021240,0.068540,0.877885,2,198.05206,93.971240,5d073b814a19c000086c558b,1560500999681
2,-7.591187,-96.911012,-104.502199,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-59.0,-77.0,7.119795,-0.127777,1.844299,7.909500,-0.095444,1.719208,7.479141,0.0,0.0,0.0,-0.352585,0.046158,-0.105713,-0.351685,0.046494,-0.104645,0.000000,0.000000,0.000000,37.811280,-3.472900,-18.107605,-44.483948,-97.181700,-352.11640,0.00000,0.00000,0.0000,0.008971,0.087430,0.808718,2,198.05206,93.971240,5d073b814a19c000086c558b,1560501001590
3,-33.353259,-88.763467,-122.116726,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-59.

In [47]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)

In [48]:
all_preds

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0.0,93.958514,99.233139
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0.0,91.940152,102.341637
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0.0,89.112067,105.326688
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0.0,89.678749,105.679917
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0.0,90.620852,110.508695
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,NaN,NaN,NaN
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,NaN,NaN,NaN
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,NaN,NaN,NaN


In [50]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)
all_preds.to_csv('submission_lgbm_28.csv')

In [51]:
!mv /content/submission_lgbm_28.csv /content/drive/MyDrive

In [46]:
floor = pd.read_csv('/content/drive/MyDrive/only_accurate_floor.csv')

In [47]:
all_preds['floor'] = floor['floor'].values

In [48]:
all_preds.to_csv('submission_lgbm_27.csv')

In [49]:
!mv /content/submission_lgbm_27.csv /content/drive/MyDrive

In [ ]:
pd.